In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/NGULIM.TTF"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)

In [5]:
path = '../data/raw/'
lst = os.listdir(path)
print(lst)

item = pd.read_csv(path+'items.csv')
category = pd.read_csv(path+'item_categories.csv')
shops = pd.read_csv(path+'shops.csv')

sample = pd.read_csv(path+'sample_submission.csv')
test = pd.read_csv(path+'test.csv')

sales_train = pd.read_csv(path+'sales_train.csv')

['items.csv', 'item_categories.csv', 'item_categories_kor.csv', 'sales_train.csv', 'sample_submission.csv', 'shops.csv', 'test.csv']


# 1. 이상치 처리
- item_cnt_day <0, item_cnt_day > 1000
- item_price > 250000

In [58]:
didx = sales_train.loc[(sales_train.item_cnt_day > 1000) | (sales_train.item_cnt_day <0) | (sales_train.item_price >250000)].index
didx.shape

(7358,)

In [59]:
print( sales_train.shape )
sales_train.drop(index=didx, inplace=True)
print( sales_train.shape)

(2935849, 6)
(2928491, 6)


# 2. Filtering Train data - by Shops in Test data

In [60]:
print(len(sales_train))
sales_train.head()

2928491


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0


In [61]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [62]:
con_shop = sales_train.shop_id.isin(test.shop_id.unique())
con_item = sales_train.item_id.isin(test.item_id.unique())

sales_train[con_shop].shape, sales_train[con_shop & con_item].shape

((2407079, 6), (1221497, 6))

In [63]:
# 샵만 필터링하여 작업.
f_sale = sales_train[con_shop].copy()
f_sale.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
5,10.01.2013,0,25,2564,349.00,1.0


# 3. Feature : 월별, 매장별, 상품별 판매량

In [64]:
# 한 날짜 안에 여러 주문 건수가 있음.
f_sale[(f_sale.date=='02.01.2013')].item_id.value_counts()

3432     26
1470     25
16450    24
2808     21
2308     21
         ..
11502     1
11468     1
10754     1
11336     1
965       1
Name: item_id, Length: 2333, dtype: int64

In [81]:
# 상품별 월평균 판매량 계산
f_sale_m = f_sale.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day':sum})
f_sale_m.columns = ['item_cnt_month']
f_sale_m.reset_index(inplace=True)
f_sale_m

,date_block_num,shop_id,item_id,item_cnt_month
0,0,2,27,1.0
1,0,2,33,1.0
2,0,2,317,1.0
3,0,2,438,1.0
4,0,2,471,2.0
...,...,...,...,...
1323024,33,59,22087,6.0
1323025,33,59,22088,2.0
1323026,33,59,22091,1.0
1323027,33,59,22100,1.0


# 4. Feature : Lag(3번 항목)

- 1,2,3,6,12월 진행

In [140]:
def lag_feature(df, lag, col):
    temp = df[['date_block_num', 'shop_id', 'item_id', col]]
    
    for i in lag:
        shift = temp.copy()
        shift.date_block_num += i
        shift.columns = ['date_block_num', 'shop_id', 'item_id', col+'_lag_'+str(i)]
        df = pd.merge(df, shift, how='left', on=['date_block_num','shop_id','item_id'])
    return df

In [141]:
f_sale_lag = lag_feature(f_sale_m, [1,2,3,6,12], 'item_cnt_month')

In [145]:
f_sale_lag.fillna(0)

,date_block_num,shop_id,item_id,item_cnt_month,item_cnt_month_lag_1,item_cnt_month_lag_2,item_cnt_month_lag_3,item_cnt_month_lag_6,item_cnt_month_lag_12
0,0,2,27,1.0,0.0,0.0,0.0,0.0,0.0
1,0,2,33,1.0,0.0,0.0,0.0,0.0,0.0
2,0,2,317,1.0,0.0,0.0,0.0,0.0,0.0
3,0,2,438,1.0,0.0,0.0,0.0,0.0,0.0
4,0,2,471,2.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
1323024,33,59,22087,6.0,3.0,2.0,5.0,2.0,6.0
1323025,33,59,22088,2.0,1.0,7.0,7.0,3.0,7.0
1323026,33,59,22091,1.0,3.0,0.0,0.0,0.0,1.0
1323027,33,59,22100,1.0,1.0,0.0,0.0,0.0,0.0
